In [55]:
%matplotlib inline

# Alternative: Extract bike parking spaces from mongoDB, export as SVG for nesting 🚲 🚲 🚲

This notebook extracts geometries (areas, like polygons of parking spaces) from a mongoDB, then exports all areas in an svg file for nesting with SVGNest. This is an alternative to be used when parkingtosvgbike fails! Only one bin is used in total for all parking spaces and spot parking spaces.

Created on:  2016-12-14  
Last update: 2016-12-14  
Contact: michael.szell@moovel.com, michael.szell@gmail.com (Michael Szell)

## Preliminaries

### Parameters

In [56]:
cityname = "vienna"

mode = "bike" # do bike here. car is another file
bikeparkw = 0.8
bikeparkh = 2

pathdatain = 'output/'+cityname+mode+'in/'
pathdataout = 'output/'+cityname+mode+'out/'

# manually excluding nodes that are tagged in OSM both as polygon and node
excludenodes = [1616515071, 1455723400]

### Imports

In [57]:
from __future__ import unicode_literals
import sys
import csv
import os
import math
from random import shuffle, choice, uniform
import random
import pprint
pp = pprint.PrettyPrinter(indent=4)
from collections import defaultdict
import time
import datetime
import numpy as np
from numpy import *
from scipy import stats
import pyprind
import itertools
import logging
from ast import literal_eval as make_tuple
from collections import OrderedDict
from retrying import retry
from copy import deepcopy

import json
from xml.dom import minidom
from shapely.geometry import mapping, shape, LineString, LinearRing, Polygon, MultiPolygon
import shapely
import shapely.ops as ops
from shapely import affinity
from functools import partial
import pyproj
Projection = pyproj.Proj("+proj=merc +lon_0=0 +x_0=0 +y_0=0 +ellps=WGS84 +units=m +no_defs")
from scipy import spatial
from haversine import haversine
import overpass
apiop = overpass.API(timeout=600)
from scipy.ndimage.interpolation import rotate
from scipy.spatial import ConvexHull

import pymongo
from pymongo import MongoClient

# plotting stuff
import matplotlib.pyplot as plt

### DB Connection

In [58]:
client = MongoClient()
db_derived = client[cityname+'_derived']
ways = db_derived['ways']
cursor = ways.find({"$and": [{"properties.amenity.amenity": "bicycle_parking"}, {"geometry.type": "Polygon"}, {"properties_derived.area": { "$gte": 1 }}]}).sort("properties_derived.area",-1)
numparkingareas = cursor.count()
print("There are " + str(numparkingareas) + " " + mode + " parking spaces in " + cityname)

db_raw = client[cityname+'_raw']
nodes = db_raw['nodes']
cursornodes = nodes.find({"$and": [{"tags.amenity.amenity": "bicycle_parking"}, { "tags.capacity.capacity": { "$exists": True }}]})
numparkingspots = cursornodes.count()
print("There are " + str(numparkingspots) + " " + mode + " parking spots in " + cityname)


There are 32 bike parking spaces in vienna
There are 1184 bike parking spots in vienna


### Functions

In [59]:
def coordinatesToSVGString(coo, xoffset = 0, yoffset = 0, idname = "", classname = "", rot = 0, centroidlatlon = [0,0]):
    svgstring = "\n  <polygon"
    if idname:
        svgstring += " id=\""+idname+"\""
    if classname:
        svgstring += " class=\""+classname+"\""
    svgstring += " points=\""
    strxylist = [str(coo[i][0]+xoffset)+","+str(coo[i][1]+yoffset) for i in range(coo.shape[0])]
    for s in strxylist:
        svgstring += s+" "
    svgstring += "\""
    svgstring += " moovel_rot=\""+str(rot)+"\"" # pseudo-namespace, because svgnest strips namespace info. http://stackoverflow.com/questions/15532371/do-svg-docs-support-custom-data-attributes
    centroid = [Polygon(coo).centroid.x, Polygon(coo).centroid.y]
    svgstring += " moovel_centroid=\""+str(centroid[0]+xoffset)+","+str(centroid[1]+yoffset)+"\""
    svgstring += " moovel_centroidlatlon=\""+str(centroidlatlon[0])+","+str(centroidlatlon[1])+"\""
    svgstring += "/>"
    return svgstring
    
def rotationToSmallestWidthRecursive(poly, maxdepth = 3, w = float("inf"), rot = 0, rotdelta = 10, depth = 1): # poly is a shapely polygon
    # unit: degrees
    # returns the angle the polygon needs to be rotated to be at minimum width
    # Note: Is not guaranteed to converge to the global minimum
    # Requires import numpy as np, from shapely import affinity
    if depth <= maxdepth:
        for theta in np.arange(rot-rotdelta*9, rot+rotdelta*9, rotdelta):
            temp = affinity.rotate(poly, theta, origin='centroid')
            x, y = temp.exterior.coords.xy
            temp = np.array([[x[i],y[i]] for i in range(len(x))])
            objectwidth = max(temp[:, 0])-min(temp[:, 0])
            if objectwidth < w:
                w = objectwidth
                rot = theta
        return rotationToSmallestWidthRecursive(poly, maxdepth, w, rot, rotdelta/10, depth+1)
    else:
        return rot

## Get parking spaces for multiple SVG bins

In [60]:
cursor = ways.find({"$and": [{"properties.amenity.amenity": "bicycle_parking"}, {"geometry.type": "Polygon"}, {"properties_derived.area": { "$gte": 1 }}]}).sort("properties_derived.area",-1)
cursornodes = nodes.find({"$and": [{"tags.amenity.amenity": "bicycle_parking"}, { "tags.capacity.capacity": { "$exists": True }}]})

random.seed(1)
scale = 0.6
erectparts = True
randomrotateparts = False
smallvsmedium = 11
buffereps = 5 # should be the same number as the distances between parts in SVGNest
height = 1200
width = 600-1.5*buffereps
eps = 0.000001

# pre-select all parts
idsused = set()
idsnotused = set()
alltiles = []
alltileskeys = []
alltilesarea = 0
areasall = []
for i,way in enumerate(cursor):
    npway = np.asarray(way["geometry"]["coordinates"])
    centroidlatlon = [Polygon(npway).centroid.x, Polygon(npway).centroid.y]
    npwayxy = [Projection(npway[i][0], npway[i][1]) for i in range(npway.shape[0])]
    npwayxy = np.asarray([[npwayxy[i][0],-npwayxy[i][1]] for i in range(npway.shape[0])])
    if erectparts:
        rot = 90+rotationToSmallestWidthRecursive(Polygon(npwayxy))
    elif randomrotateparts:
        rot = uniform(10, 350)
    else:
        rot = 0
    if rot:
        temp = affinity.rotate(Polygon(npwayxy), rot, origin='centroid', use_radians=False)
        x, y = temp.exterior.coords.xy
        npwayxy = np.array([[x[i],y[i]] for i in range(len(x))])
    objectwidth = max(npwayxy[:, 0])-min(npwayxy[:, 0])
    npwayxy[:, 0] -= min(npwayxy[:, 0])
    npwayxy[:, 1] -= min(npwayxy[:, 1])
    npwayxy *= scale
    objectwidth *= scale
    if objectwidth < width:
        objectheight = max(npwayxy[:, 1])
        idsnotused.add(int(way["_id"]))
        coo = [[npwayxy[k][0], npwayxy[k][1]] for k in range(npwayxy.shape[0])]
        areasall.append(Polygon(coo).area)
        area = Polygon(coo).buffer(buffereps/2).area
        alltiles.append( { "_id": int(way["_id"]), "width": objectwidth, "height": objectheight, "area": area, "coordinates": coo , "rot": rot, "centroidlatlon": centroidlatlon})
        alltileskeys.append(int(way["_id"]))
        alltilesarea += area
    else:
        print("Object "+str(way["_id"])+" was too wide (" +str(objectwidth)+ " pixel) and was ignored.")
        
# Generation of polygons from point parking
capacitiesall = []
for i,node in enumerate(cursornodes):
    try: # sometimes capacity is not an integer
        capacity = int(node["tags"]["capacity"]["capacity"])
    except:
        capacity = 0
    if capacity and node["_id"] not in excludenodes:
        centroidlatlon = node["loc"]["coordinates"]
        if capacity <= 20:
            xd = capacity*bikeparkw/2
            yd = bikeparkh/2
        else:
            xd = math.sqrt(capacity)*bikeparkh/2
            yd = math.sqrt(capacity)*bikeparkw/2
        npwayxy = [[-xd, -yd], [xd, -yd], [xd, yd], [-xd, yd]]
        npwayxy = np.asarray([[npwayxy[i][0],-npwayxy[i][1]] for i in range(4)])
        objectwidth = max(npwayxy[:, 0])-min(npwayxy[:, 0])
        npwayxy[:, 0] -= min(npwayxy[:, 0])
        npwayxy[:, 1] -= min(npwayxy[:, 1])
        npwayxy *= scale
        objectwidth *= scale
        if objectwidth < width:
            objectheight = max(npwayxy[:, 1])
            idsnotused.add(int(node["_id"]))
            coo = [[npwayxy[k][0], npwayxy[k][1]] for k in range(npwayxy.shape[0])]
            areasall.append(Polygon(coo).area)
            area = Polygon(coo).buffer(buffereps/2).area
            alltiles.append( { "_id": int(node["_id"]), "width": objectwidth, "height": objectheight, "area": area, "coordinates": coo , "rot": rot, "centroidlatlon": centroidlatlon})
            alltileskeys.append(int(node["_id"]))
            alltilesarea += area
            capacitiesall.append(capacity)
        else:
            print("Object "+str(node["_id"])+" was too wide (" +str(objectwidth)+ " pixel) and was ignored.")
sortind = [i[0] for i in sorted(enumerate(areasall), key=lambda x:x[1], reverse=True)]

# Parking spaces and spots in one
bigbin = Polygon([[0,0], [width,0], [width, height], [0, height]])
bigbinarea = bigbin.area
# change the big bin area according to the tiles area
heightbigbin = 1.28 * height * alltilesarea/bigbinarea
bigbin = Polygon([[0,0], [width,0], [width, heightbigbin], [0, heightbigbin]])     
# Fill with parts
binbound = np.array(bigbin.exterior.coords)
xpos = 0
ypos = 0
yextent = 0
svg = "<svg xmlns=\"http://www.w3.org/2000/svg\" version=\"1.1\" width=\""+str(width)+"px\" height=\""+str(heightbigbin)+"px\">"
cnt = 0
for j in sortind:
    if len(idsnotused) == 0:
        break
    tile = alltiles[j]
    if tile["width"] <= width:
        if xpos + tile["width"] + 1 <= width: # there is space in this row
            xdelta = (xpos+1)
            ydelta = ypos
        else: # new row
            xdelta = 0
            ypos += yextent+buffereps
            yextent = 0
            ydelta = ypos
            xpos = 0
        svg += coordinatesToSVGString(np.array([[tile["coordinates"][k][0], tile["coordinates"][k][1]] for k in range(np.array(tile["coordinates"]).shape[0])]), xdelta, ydelta, str(tile["_id"]), "tile", tile["rot"], tile["centroidlatlon"])
        yextent = max([yextent, tile["height"]])
        xpos += tile["width"]+buffereps
        idsused.add(tile["_id"])
        idsnotused.remove(tile["_id"])
        cnt += 1
    else:
        print("Object "+str(way["_id"])+" was too wide (" +str(max(npwayxy[:, 0]))+ " pixel) and could not be placed.")
svg += "\n</svg>"
with open(pathdataout + "all.svg", "w") as f:
    f.write(svg)
    

print("Export done. " + str(len(idsnotused))+" tiles were not used.")

Export done. 0 tiles were not used.


The result is a file in {{pathdataout}}. No use of SVGNest required.